# Publish Fairness LoRA Adapter to Hugging Face

This notebook exports the fairness-aware LoRA adapter from the trained checkpoint, writes a Model Card (README.md), and optionally pushes the adapter folder to the Hugging Face Hub.

Steps:
- Build the fair LoRA model and load the checkpoint.
- Save only the PEFT adapter to `models/lora_adapters/fairness_lora`.
- Auto-generate a Model Card with metrics (if available).
- Optionally push the adapter repo to the Hub.

In [2]:
# Imports and project path setup
from pathlib import Path
import sys, os
import json
import torch

# Resolve project root robustly (handles notebook working dir variance)
nb_dir = Path.cwd()
# Candidate roots to add: current dir, parent, and repo root if notebooks/ layout
candidates = [
    nb_dir,
    nb_dir.parent,
    nb_dir.parent.parent,
]
for c in candidates:
    if str(c) not in sys.path:
        sys.path.insert(0, str(c))

# Import helper functions from the export script
try:
    from scripts.export_fair_lora_to_hf import (
        build_model,
        load_checkpoint_into_model,
        maybe_collect_metrics,
        write_model_card,
        push_to_hub,
    )
except ModuleNotFoundError as e:
    # As a fallback, try to import by absolute path based on repo structure
    repo_root = None
    for c in candidates:
        if (c / 'scripts' / 'export_fair_lora_to_hf.py').exists():
            repo_root = c
            break
    if repo_root and str(repo_root) not in sys.path:
        sys.path.insert(0, str(repo_root))
    from scripts.export_fair_lora_to_hf import (
        build_model,
        load_checkpoint_into_model,
        maybe_collect_metrics,
        write_model_card,
        push_to_hub,
    )

print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())

/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


Torch version: 2.7.1
CUDA available: False


In [27]:
# Configuration
try:
    from fair_lora_config import BASE_MODEL as CONFIG_BASE_MODEL
except Exception:
    CONFIG_BASE_MODEL = 'BAAI/bge-large-en-v1.5'

BASE_MODEL = CONFIG_BASE_MODEL  # Base encoder model on which the LoRA adapter sits
CHECKPOINT = '../models/fair_adversarial/best_fairness_model.pt'  # Trained fairness model checkpoint
ADAPTER_DIR = '../models/peft_adapters/fairness_lora'  # Where to save the adapter locally (avoid clashing with any file)
REPO_ID = 'renhehuang/fair-resume-job-matcher-lora'  # e.g., 'your-username/fair-resume-matcher-lora' when pushing to Hub
PRIVATE = False  # If pushing, create a private repo if True
DO_PUSH = True  # Set True to push to Hub (requires REPO_ID)

# Resolve repo root and make paths absolute for robustness
_nb_dir = Path.cwd()
_candidates = [_nb_dir, _nb_dir.parent, _nb_dir.parent.parent]
_repo_root = None
for c in _candidates:
    if (c / 'scripts' / 'export_fair_lora_to_hf.py').exists():
        _repo_root = c
        break
_repo_root = _repo_root or _nb_dir

ckpt_path = Path(CHECKPOINT)
if not ckpt_path.is_absolute():
    ckpt_path = (_repo_root / ckpt_path).resolve()
adapter_dir = Path(ADAPTER_DIR)
if not adapter_dir.is_absolute():
    adapter_dir = (_repo_root / adapter_dir).resolve()

print('Base model:', BASE_MODEL)
print('Checkpoint:', ckpt_path)
print('Adapter out dir:', adapter_dir)
print('Repo id (optional):', REPO_ID or '(not set)')

Base model: BAAI/bge-large-en-v1.5
Checkpoint: /Users/edwardhuang/Documents/GitHub/models/fair_adversarial/best_fairness_model.pt
Adapter out dir: /Users/edwardhuang/Documents/GitHub/models/peft_adapters/fairness_lora
Repo id (optional): renhehuang/fair-resume-job-matcher-lora


In [23]:
# Build the model, load checkpoint, and save only the PEFT adapter
out_dir = adapter_dir
out_dir.mkdir(parents=True, exist_ok=True)

model = build_model(BASE_MODEL)
_missing, _unexpected = load_checkpoint_into_model(model, ckpt_path)

# Save the PEFT adapter (LoRA weights)
model.base_model.save_pretrained(out_dir)
print(f'\u2713 Saved PEFT adapter to: {out_dir}')

# List saved files
print('Saved files:')
for p in sorted(out_dir.glob('*')):
    print(' -', p.name)

🔧 Loading base model: BAAI/bge-large-en-v1.5
🔧 Applying LoRA (expanded targets, r=16, alpha=32)...
   • LoRA target modules detected: ['dense', 'key', 'query', 'value']
     (modules not present are ignored silently by PEFT)
🔧 Applying LoRA (expanded targets, r=16, alpha=32)...
   • LoRA target modules detected: ['dense', 'key', 'query', 'value']
     (modules not present are ignored silently by PEFT)
🔒 Frozen encoder base weights except attention in last 4 layers; LoRA adapters remain trainable.
📊 Trainable params -> LoRA: 7,110,656 | Base last4(attn): 16,801,792 | Total: 23,912,448
trainable params: 23,912,448 || all params: 342,252,544 || trainable%: 6.9868
🔧 Adding adversarial discriminator...
🔧 Adding attribute classifier for multi-task learning...
🔒 Frozen encoder base weights except attention in last 4 layers; LoRA adapters remain trainable.
📊 Trainable params -> LoRA: 7,110,656 | Base last4(attn): 16,801,792 | Total: 23,912,448
trainable params: 23,912,448 || all params: 342,25

Repo card metadata block was not found. Setting CardData to empty.


Loaded checkpoint: /Users/edwardhuang/Documents/GitHub/bge-lora-fairness-finetuning/models/fair_adversarial/best_fairness_model.pt
  Missing keys: 0 | Unexpected keys: 0
✓ Saved PEFT adapter to: /Users/edwardhuang/Documents/GitHub/bge-lora-fairness-finetuning/models/peft_adapters/fairness_lora
Saved files:
 - README.md
 - adapter_config.json
 - adapter_model.safetensors
✓ Saved PEFT adapter to: /Users/edwardhuang/Documents/GitHub/bge-lora-fairness-finetuning/models/peft_adapters/fairness_lora
Saved files:
 - README.md
 - adapter_config.json
 - adapter_model.safetensors


In [29]:
# Generate Model Card (README.md) with metrics if available
import importlib, scripts.export_fair_lora_to_hf as exp
importlib.reload(exp)
from scripts.export_fair_lora_to_hf import write_model_card, maybe_collect_metrics

metrics_csv = (_repo_root / 'models' / 'fair_adversarial' / 'epoch_metrics.csv').resolve()
metrics = maybe_collect_metrics(metrics_csv)
print('Collected metrics:', json.dumps(metrics, indent=2))

# Use repo id if provided; else fallback to folder name
repo_for_card = REPO_ID or adapter_dir.name
write_model_card(adapter_dir, repo_for_card, BASE_MODEL, 'fairness-lora', metrics)

readme_path = adapter_dir / 'README.md'
print('Model Card written to:', readme_path)
print('\n--- README preview ---')
preview = readme_path.read_text(encoding='utf-8')
print(preview[:1000])

Collected metrics: {}
Model Card written to: /Users/edwardhuang/Documents/GitHub/models/peft_adapters/fairness_lora/README.md

--- README preview ---
---
tags:
  - lora
  - peft
  - fairness
  - resume-matching
  - retrieval
  - sentence-similarity
library_name: peft
base_model: BAAI/bge-large-en-v1.5
pipeline_tag: sentence-similarity
language:
  - en
---

# renhehuang/fair-resume-job-matcher-lora

Fairness-aware LoRA adapter for resume–job matching built on top of `BAAI/bge-large-en-v1.5`.

This adapter was trained with adversarial debiasing and multi-task objectives to reduce group disparities while maintaining utility.

## Model Summary
- Base model: `BAAI/bge-large-en-v1.5`
- Adapter type: LoRA (PEFT)
- Task: Resume–job text similarity (cosine over mean-pooled, L2-normalized embeddings; optional sigmoid for probability)
- Intended audience: Researchers and practitioners exploring fairness-aware matching

## Quick Start

```python
from transformers import AutoTokenizer, AutoModel
fr

## Optional: Login and prepare to push
- Fill `REPO_ID` in the config cell (e.g., `your-username/fair-resume-matcher-lora`).
- Toggle `DO_PUSH = True` if you want to upload.
- If you haven't logged in from this environment, run the cell below to login via a widget.

In [ ]:
from huggingface_hub import login
login(token='')  # Uncomment to login via a widget in notebook environments


In [28]:
# Push to Hugging Face Hub (if enabled)
if DO_PUSH:
    if not REPO_ID:
        raise ValueError('DO_PUSH is True but REPO_ID is empty. Please set REPO_ID first.')
    push_to_hub(Path(ADAPTER_DIR), REPO_ID, private=PRIVATE)
else:
    print('Skipping push (set DO_PUSH=True to upload).')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Uploading...:   0%|          | 0.00/28.5M [00:00<?, ?B/s]

✓ Uploaded adapter to https://huggingface.co/renhehuang/fair-resume-job-matcher-lora


In [14]:
# Smoke test: load adapter locally and compute a cosine score
from transformers import AutoModel, AutoTokenizer
from peft import PeftModel
import torch.nn.functional as F

adapter_path = str(adapter_dir)
print('Using adapter at:', adapter_path)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
base = AutoModel.from_pretrained(BASE_MODEL)
model = PeftModel.from_pretrained(base, adapter_path)
model.eval()

def encode(text: str):
    enc = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=256)
    with torch.no_grad():
        out = model(**enc)
        emb = F.normalize(out.last_hidden_state.mean(dim=1), p=2, dim=1)
    return emb

a = 'Software engineer with Python experience.'
b = 'Hiring backend Python developer with API experience.'
cos = (encode(a) * encode(b)).sum(dim=1).item()
prob = torch.sigmoid(torch.tensor(cos)).item()
print({'cosine': cos, 'prob': prob})

Using adapter at: /Users/edwardhuang/Documents/GitHub/bge-lora-fairness-finetuning/models/peft_adapters/fairness_lora
{'cosine': 0.8230911493301392, 'prob': 0.6948921084403992}
{'cosine': 0.8230911493301392, 'prob': 0.6948921084403992}
